CREADOR DE TABLA MAESTRA

Este notebook fue utilizada para crear un único archivo csv fácil de procesar en Looker Studio.

In [ ]:
import pandas as pd

#PREPROCESAMIENTO
# Preparar df's
df_orders = pd.read_csv('../data/raw/order_all.csv')
df_items = pd.read_csv('../data/raw/order_items.csv')
df_products = pd.read_csv('../data/raw/products.csv')
df_customers = pd.read_csv('../data/raw/customers.csv')
df_translations = pd.read_csv('../data/raw/translations.csv')


#JOINS
#Orders + items
master = df_orders.merge(df_items, on='order_id', how='left')

#+ clientes
master = master.merge(df_customers, on='customer_id', how='left')

#+ productos
master = master.merge(df_products, on='product_id', how='left')

#+ traducciones
master = master.merge(df_translations, on='product_category_name', how='left')


#LIMPIEZA
#Filtrar entregados
master = master[master['order_status'] == 'delivered']

#Rellenar nulos en traducciones
#Si no hay traducción, usamos el nombre en portugués. Si no hay nada, 'Otros'.
master['product_category_name_english'] = master['product_category_name_english'].fillna(master['product_category_name'])
master['product_category_name_english'] = master['product_category_name_english'].fillna('Others')

#Arreglar formato de fecha
master['order_purchase_timestamp'] = pd.to_datetime(master['order_purchase_timestamp'])

# Seleccionar columnas principales
columnas_finales = [
    'order_id',                     # Key para distinguir pedidos
    'order_purchase_timestamp',     # Análisis temporal
    'price',                        # Medir rentabilidad
    'freight_value',                # Costos de envío
    'customer_state',               # Análisis geográfico
    'customer_city',                # Análisis geográfico
    'product_category_name_english' # Para labels
]

df_dashboard = master[columnas_finales]


#GUARDADO
df_dashboard.to_csv('../data/processed/ecommerce_master_table.csv', index=False)

print(f"Tabla maestra creada con {df_dashboard.shape[0]} filas.")